In [6]:
from numpy import mean
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import SVC
from skopt.space import Integer
from skopt.space import Real
from skopt.space import Categorical
from skopt.utils import use_named_args
from skopt import gp_minimize
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor

# define the space of hyperparameters to search
search_space = list()
search_space.append(Integer(10, 500, name='n_estimators'))
search_space.append(Integer(1,10, name='min_samples_leaf'))
search_space.append(Integer(1, 10, name='max_depth'))
search_space.append(Integer(1, 30, name='max_features'))

# define the function used to evaluate a given configuration
@use_named_args(search_space)
def evaluate_model(**params):
    # configure the model with specific hyperparameters
    model = RandomForestRegressor()
    model.set_params(**params)
    # define test harness
    #cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=1)
    # calculate 5-fold cross validation
    result = cross_val_score(model, X, y, cv=10, n_jobs=-1, scoring='r2')
    # calculate the mean of the scores
    estimate = mean(result)
    # convert from a maximizing score to a minimizing score
    return 1.0 - estimate

# load dataset
dataset = pd.read_csv('fullset_with_more_fields.csv')
for column in dataset.columns:
    dataset = dataset.drop(dataset[dataset[str(column)] == 'ERROR'].index)
dataset['author_type'] = dataset.author_type.apply(lambda x: 1 if x=='User' else 0)
X = dataset.drop(['stars'] , axis =1)

y = dataset.stars
s = StandardScaler()
X = s.fit_transform(X)
# perform optimization
result = gp_minimize(evaluate_model, search_space)
# summarizing finding:
print('Best Accuracy: %.3f' % (1.0 - result.fun))
print('Best Parameters: %s' % (result.x))

c:\Users\jonat\anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\jonat\anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\jonat\anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Best Accuracy: -350.226
Best Parameters: [395, 4, 10, 14]


In [28]:
from sklearn.model_selection import KFold
dataset = pd.read_csv('fullset_with_more_fields.csv')

for column in dataset.columns:
    dataset = dataset.drop(dataset[dataset[str(column)] == 'ERROR'].index)
dataset['author_type'] = dataset.author_type.apply(lambda x: 1 if x=='User' else 0)


X = dataset.drop(['stars'] , axis =1)

y = dataset.stars

#s = StandardScaler()
#X = s.fit_transform(X)
print(type(X))
kf = KFold(n_splits=5)
kf.get_n_splits(X)
print(kf)
acc_score = []
for i, (train_index, test_index) in enumerate(kf.split(X)):
    print(f"Fold {i}:")
    print(f"  Train: index={train_index}")
    print(f"  Test:  index={test_index}")

    clf = RandomForestRegressor(n_jobs=-1, n_estimators=200, random_state=42)
    print(y)
    clf.fit(X.iloc[train_index], y.iloc[train_index])

    test_score = r2_score(y.iloc[test_index], clf.predict(X.iloc[test_index]))
    acc_score.append(test_score)

avg_acc_score = sum(acc_score)/5
 
print('accuracy of each fold - {}'.format(acc_score))
print('Avg accuracy : {}'.format(avg_acc_score))

<class 'pandas.core.frame.DataFrame'>
KFold(n_splits=5, random_state=None, shuffle=False)
Fold 0:
  Train: index=[200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217
 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235
 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253
 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271
 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286 287 288 289
 290 291 292 293 294 295 296 297 298 299 300 301 302 303 304 305 306 307
 308 309 310 311 312 313 314 315 316 317 318 319 320 321 322 323 324 325
 326 327 328 329 330 331 332 333 334 335 336 337 338 339 340 341 342 343
 344 345 346 347 348 349 350 351 352 353 354 355 356 357 358 359 360 361
 362 363 364 365 366 367 368 369 370 371 372 373 374 375 376 377 378 379
 380 381 382 383 384 385 386 387 388 389 390 391 392 393 394 395 396 397
 398 399 400 401 402 403 404 405 406 407 408 409 410 411 412 413 414 415
 416 417 41